In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!git clone https://github.com/fizyr/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [5]:
cd keras-retinanet/

/content/keras-retinanet


In [6]:
!pip install .

Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=169825 sha256=b8f4fc226cc4fc7434aa824fb5e751d1a67395203d8b921a653107ea9bece075
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13344 sha256=2c040b112fcf38d098addb081433a9001bcd8b024181c4d7cdc166805cbf3dfc
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [7]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [0]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files

In [0]:
DATASET_DRIVEID = '163TTCVg2eoxM20qakU56tVcjbhTvttdi'

In [0]:
DATASET_DIR = 'dataset'
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

In [0]:
drive_url = 'https://drive.google.com/uc?export=download&id=' + DATASET_DRIVEID
file_name = DATASET_DRIVEID + '.zip'

urllib.request.urlretrieve(drive_url, file_name)
print('Download completed!')

os.makedirs(DATASET_DIR, exist_ok=True)
with zipfile.ZipFile(file_name, 'r') as zip_ref:
  zip_ref.extractall(DATASET_DIR)
os.remove(file_name)
print('Extract completed!')

Download completed!
Extract completed!


In [0]:
annotations = []
classes = set([])

for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  file_name = None

  for elem in root:
    if elem.tag == 'filename':
      file_name = os.path.join(DATASET_DIR, elem.text)

    if elem.tag == 'object':
      obj_name = None
      coords = []
      for subelem in elem:
        if subelem.tag == 'name':
          obj_name = subelem.text
        if subelem.tag == 'bndbox':
          for subsubelem in subelem:
            coords.append(subsubelem.text)
      item = [file_name] + coords + [obj_name]
      annotations.append(item)
      classes.add(obj_name)

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))

In [12]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

#### OPTION 2: DOWNLOAD CUSTOM PRETRAINED MODEL FROM GOOGLE DRIVE. CHANGE DRIVE_MODEL VALUE. USE THIS TO CONTINUE PREVIOUS TRAINING EPOCHS ####
#drive.mount('/content/gdrive')
#DRIVE_MODEL = '/content/gdrive/My Drive/Colab Notebooks/objdet_tensorflow_colab/resnet50_csv_10.h5'
#shutil.copy(DRIVE_MODEL, PRETRAINED_MODEL)


print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [13]:
!pip3 install keras==2.3.0

     |████████████████████████████████| 378kB 3.5MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 250 --epochs 5 csv annotations.csv classes.csv

Using TensorFlow backend.
Creating model, this may take a second...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1316: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 9) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1316: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((9,) vs (720,)).
  weight_values[i].shape))
2019-12-07 08:46:40.110538: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-07 08:46:40.132899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-12-07 08:46:40.133712: I tensorflow/co

In [14]:
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [15]:
model_path = '/content/drive/My Drive/resnet50_csv_10.h5'
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv(CLASSES_FILE,header=None).T.loc[0].to_dict()

/content/drive/My Drive/resnet50_csv_10.h5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


In [18]:
import json
import os
import pickle as pkl

path_file = '/content/anh/'

def img_inference(img_path, pos_window):
    window_size = (240, 250)
    over_lap_x = 100
    over_lap_y = 80
    list_bb = []
    image = read_image_bgr(img_path)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    # print(scale)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    boxes /= scale
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        box_restore = []
        # scores are sorted so we can break
        if score < 0.85 or labels_to_names[label] != "solar":
            break
        
        box_restore.append(box[0] + pos_window[1] * (window_size[1]- over_lap_y))
        box_restore.append(box[1] + pos_window[0] * (window_size[0]- over_lap_x))
        box_restore.append(box[2] + pos_window[1] * (window_size[1]- over_lap_y))
        box_restore.append(box[3] + pos_window[0] * (window_size[0]- over_lap_x))
        # print(box)
        # print(score)
        # print(labels_to_names[label])
        list_bb.append(box_restore)
        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()
    # for box in list_bb:
    #     print(box)
    # print(list_bb[0])
    return list_bb
list_bb = {}
for img in os.listdir(path_file):
    A = img.split('.')
    pos_window = (int(A[0]), int(A[1]))
    path_img = os.path.join(path_file, img)
    id = A[0] + '.' + A[1]
    print(id)
    bb = img_inference(path_img, pos_window)
    # print(bb)
    list_bb[id] = bb

with open('./list_bb.json', 'w') as f_w:
    json.dump(list_bb, f_w)
print(len(list_bb))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import cv2
import numpy as np


def sliding_window(path_img):
    over_lap_x = 100
    over_lap_y = 80
    list_img = []
    list_solar = []
    window_size = (240, 250)
    img = cv2.imread(path_img)
    # print(np.shape(img))
    x, y,_ = np.shape(img)
    # print(x, y)
    x = x//(window_size[0] - over_lap_x)
    y = y//(window_size[1] - over_lap_y)
    print(x, y)
    image_size_x = (window_size[0] - over_lap_x) * x + over_lap_x
    image_size_y = (window_size[1] - over_lap_y) * y + over_lap_y
    print(image_size_x, image_size_y)
    image_size = (image_size_y, image_size_x)
    image = cv2.resize(img, (image_size))
    # print(np.shape(image))
    for i in range(x):
        for j in range(y):
            if(i != 0 and j != 0):
                split_image = image[(window_size[0]-over_lap_x)*i :(window_size[0]-over_lap_x)*i + window_size[0], (window_size[1]-over_lap_y)*j : (window_size[1]-over_lap_y)*j + window_size[1]]
            if(i == 0 and j != 0):
                split_image = image[0 : window_size[0], (window_size[1]-over_lap_y)*j : (window_size[1]-over_lap_y)*j + window_size[1]]
            if(j == 0 and i != 0):
                split_image = image[(window_size[0]-over_lap_x)*i :(window_size[0]-over_lap_x)*i + window_size[0], 0 : window_size[1]]
            if(i == 0 and j == 0):
                split_image = image[0: window_size[0], 0 : window_size[1]]
            # list_img.append(split_image)
            cv2.imwrite('/content/anh/' + str(i)+'.'+str(j) + '.jpg', split_image)
    cv2.imwrite('/content/a.jpg', image)
    # id = 0
    # for split_image in list_img:
        # id += 1
    # print(np.shape(list_img[0]))
    # for split_image in list_img:
    #     list_solar.extend(img_inference(split_image))

# sliding_window('/home/hung-vt/Pictures/dataset/7fc8992d8a_012288112DOPENPIPELINE_Orthomosaic_export_FriNov22014645.383588.jpg')

In [17]:
sliding_window('/content/7fc8992d8a_012288112DOPENPIPELINE_Orthomosaic_export_FriNov22014645.383588.jpg')

20 32
2900 5520


In [0]:
rm -r /content/anh1 /content/anh2 /content/anh3

In [0]:
img_inference('/content/86.jpg')

NameError: ignored

In [0]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13022       10827         183         278        2011        2000
Swap:             0           0           0


In [0]:
!python3